# Two-Stage Recommendation System
## Retrieval (Matrix Factorization) + Ranking (XGBoost)

This notebook implements a two-stage recommendation system:
1. **Retrieval Stage**: Factorization Machines to generate embeddings and retrieve top-100 candidates
2. **Ranking Stage**: XGBoost to rank candidates using rich metadata from Feature Store


## 1. Initialization and Configuration

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import json
import io
import time
from datetime import datetime
from typing import List, Dict, Tuple
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.feature_store import FeatureStore
from sagemaker import image_uris
import sagemaker.amazon.common as smac
from sklearn.neighbors import NearestNeighbors
import pickle
import os


In [ ]:
# Configuration
region = "ap-south-1"
role_arn = "arn:aws:iam::487512486150:role/recommendationsystem-sagemaker-role"
bucket = "amazon-sagemaker-local-dev-store"

# Feature Group Names (Update these with your actual Feature Group names)
USER_FEATURE_GROUP_NAME = "all-beauty-users-<timestamp>"  # Replace with actual name
ITEM_FEATURE_GROUP_NAME = "all-beauty-items-<timestamp>"  # Replace with actual name

# Initialize sessions
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(
    boto_session=boto_session,
    default_bucket=bucket
)

# Feature Store clients
featurestore_runtime = boto_session.client(
    service_name='sagemaker-featurestore-runtime',
    region_name=region
)
sagemaker_client = boto_session.client(
    service_name='sagemaker',
    region_name=region
)

feature_store = FeatureStore(sagemaker_session=sagemaker_session)

print(f"Initialized SageMaker session in {region}")
print(f"Default bucket: {bucket}")


## 2. Retrieval Stage: Matrix Factorization with Factorization Machines

### 2.1 Prepare Training Data for Factorization Machines


In [ ]:
def prepare_fm_training_data(
    interactions_df: pd.DataFrame,
    output_s3_path: str
) -> Tuple[str, Dict, int]:
    """
    Prepare training data for Factorization Machines.
    
    Factorization Machines requires a specific format:
    - Each line: label |user_id |item_id
    - user_id and item_id are categorical features (one-hot encoded)
    
    Args:
        interactions_df: DataFrame with columns [user_id, parent_asin, rating]
        output_s3_path: S3 path to save the training data
    
    Returns:
        Tuple of (s3_uri, mappings_dict, feature_dim)
    """
    # Create mappings for user_id and parent_asin to indices
    unique_users = interactions_df['user_id'].unique()
    unique_items = interactions_df['parent_asin'].unique()
    
    user_to_idx = {user: idx for idx, user in enumerate(unique_users)}
    item_to_idx = {item: idx for idx, item in enumerate(unique_items)}
    
    # Calculate feature dimensions
    num_users = len(unique_users)
    num_items = len(unique_items)
    feature_dim = num_users + num_items  # Total one-hot dimensions
    
    print(f"Number of unique users: {num_users}")
    print(f"Number of unique items: {num_items}")
    print(f"Feature dimension: {feature_dim}")
    
    # Convert to sparse format for FM
    # Format: label |user_feature_index |item_feature_index
    records = []
    for _, row in interactions_df.iterrows():
        user_idx = user_to_idx[row['user_id']]
        item_idx = item_to_idx[row['parent_asin']] + num_users  # Offset by num_users
        rating = row['rating']
        
        # FM format: label |feature_index:value
        record = f"{rating} |{user_idx}:1.0 |{item_idx}:1.0"
        records.append(record)
    
    # Save to local file first
    local_file = '/tmp/fm_training_data.txt'
    with open(local_file, 'w') as f:
        f.write('\n'.join(records))
    
    # Upload to S3
    s3_client = boto_session.client('s3')
    s3_path = output_s3_path.replace(f's3://{bucket}/', '')
    s3_client.upload_file(local_file, bucket, s3_path)
    
    s3_uri = f"s3://{bucket}/{s3_path}"
    print(f"Training data saved to: {s3_uri}")
    
    # Save mappings for later use
    mappings = {
        'user_to_idx': user_to_idx,
        'item_to_idx': item_to_idx,
        'idx_to_user': {idx: user for user, idx in user_to_idx.items()},
        'idx_to_item': {idx: item for item, idx in item_to_idx.items()},
        'num_users': num_users,
        'num_items': num_items,
        'feature_dim': feature_dim
    }
    
    # Save mappings to S3
    mappings_key = s3_path.replace('training_data.txt', 'mappings.pkl')
    mappings_buffer = io.BytesIO()
    pickle.dump(mappings, mappings_buffer)
    mappings_buffer.seek(0)
    s3_client.upload_fileobj(mappings_buffer, bucket, mappings_key)
    
    return s3_uri, mappings, feature_dim


### 2.2 Train Factorization Machines Model


In [ ]:
def train_factorization_machines(
    training_data_s3_uri: str,
    feature_dim: int,
    num_factors: int = 64,
    epochs: int = 10,
    instance_type: str = 'ml.c5.xlarge'
) -> sagemaker.estimator.Estimator:
    """
    Train a Factorization Machines model for collaborative filtering.
    
    Args:
        training_data_s3_uri: S3 URI of training data
        feature_dim: Dimension of feature space (num_users + num_items)
        num_factors: Number of latent factors
        epochs: Number of training epochs
        instance_type: SageMaker instance type for training
    
    Returns:
        Trained SageMaker estimator
    """
    # Get Factorization Machines container
    container = image_uris.retrieve(
        "factorization-machines",
        boto_session.region_name,
        version='1'
    )
    
    # Create estimator
    fm_estimator = sagemaker.estimator.Estimator(
        container,
        role=role_arn,
        instance_count=1,
        instance_type=instance_type,
        output_path=f"s3://{bucket}/fm-model-artifacts/",
        sagemaker_session=sagemaker_session
    )
    
    # Set hyperparameters
    fm_estimator.set_hyperparameters(
        feature_dim=feature_dim,
        predictor_type='regressor',  # Predicting rating (continuous value)
        num_factors=num_factors,     # Latent factors for embeddings
        epochs=epochs
    )
    
    # Start training
    print("Starting Factorization Machines training...")
    fm_estimator.fit({'train': training_data_s3_uri})
    
    print(f"Training complete! Model artifacts: {fm_estimator.model_data}")
    
    return fm_estimator


### 2.3 Extract User and Item Embeddings

**Note**: Factorization Machines models store embeddings in the weight matrix. We need to download the model artifacts and extract the V (factorization) matrix which contains the embeddings.


In [ ]:
def extract_embeddings(
    fm_estimator: sagemaker.estimator.Estimator,
    mappings: Dict,
    num_factors: int = 64
) -> Tuple[np.ndarray, np.ndarray, Dict[str, np.ndarray]]:
    """
    Extract user and item embeddings from trained FM model.
    
    The FM model learns latent factors (embeddings) for each user and item.
    We need to download the model artifacts and extract the weight matrices.
    
    Args:
        fm_estimator: Trained FM estimator
        mappings: Dictionary with user/item mappings
        num_factors: Number of latent factors
    
    Returns:
        Tuple of (user_embeddings, item_embeddings, embedding_dict)
    """
    import tarfile
    try:
        import mxnet as mx
    except ImportError:
        print("Installing mxnet...")
        import subprocess
        subprocess.check_call(["pip", "install", "mxnet"])
        import mxnet as mx
    
    # Download model artifacts
    model_uri = fm_estimator.model_data
    local_model_path = '/tmp/fm_model.tar.gz'
    
    s3_client = boto_session.client('s3')
    # Parse S3 URI
    s3_path = model_uri.replace('s3://', '')
    bucket_name, key = s3_path.split('/', 1)
    
    print(f"Downloading model from {model_uri}...")
    s3_client.download_file(bucket_name, key, local_model_path)
    
    # Extract tar.gz
    with tarfile.open(local_model_path, 'r:gz') as tar:
        tar.extractall('/tmp/fm_model')
    
    # Load MXNet model (FM uses MXNet)
    model_path = '/tmp/fm_model/model'
    sym, arg_params, aux_params = mx.model.load_checkpoint(model_path, 0)
    
    # Extract weight matrices
    # FM model structure: linear weights (w) and factorized weights (V)
    # V matrix contains the embeddings: shape (feature_dim, num_factors)
    
    # Find the V (embedding) matrix in the parameters
    v_key = None
    for key in arg_params.keys():
        if 'v' in key.lower() or 'factor' in key.lower():
            v_key = key
            break
    
    if v_key is None:
        # Fallback: assume standard FM parameter naming
        v_key = 'v'
    
    # Get embedding matrix
    V = arg_params[v_key].asnumpy()  # Shape: (feature_dim, num_factors)
    
    num_users = mappings['num_users']
    num_items = mappings['num_items']
    
    # Split V into user and item embeddings
    user_embeddings = V[:num_users, :]  # Shape: (num_users, num_factors)
    item_embeddings = V[num_users:num_users+num_items, :]  # Shape: (num_items, num_factors)
    
    print(f"User embeddings shape: {user_embeddings.shape}")
    print(f"Item embeddings shape: {item_embeddings.shape}")
    
    # Create lookup dictionaries
    embedding_dict = {}
    
    # User embeddings
    for user_id, idx in mappings['user_to_idx'].items():
        embedding_dict[f"user_{user_id}"] = user_embeddings[idx]
    
    # Item embeddings
    for item_id, idx in mappings['item_to_idx'].items():
        embedding_dict[f"item_{item_id}"] = item_embeddings[idx]
    
    # Save embeddings to S3 for later use
    s3_client = boto_session.client('s3')
    embeddings_s3_key = f"fm-embeddings/user_embeddings.npy"
    items_embeddings_s3_key = f"fm-embeddings/item_embeddings.npy"
    
    # Save numpy arrays
    user_buffer = io.BytesIO()
    np.save(user_buffer, user_embeddings)
    user_buffer.seek(0)
    s3_client.upload_fileobj(user_buffer, bucket, embeddings_s3_key)
    
    item_buffer = io.BytesIO()
    np.save(item_buffer, item_embeddings)
    item_buffer.seek(0)
    s3_client.upload_fileobj(item_buffer, bucket, items_embeddings_s3_key)
    
    print(f"Embeddings saved to S3")
    
    return user_embeddings, item_embeddings, embedding_dict


### 2.4 Top-100 K-Nearest Neighbors Search


In [ ]:
def build_knn_index(
    item_embeddings: np.ndarray,
    item_mappings: Dict,
    n_neighbors: int = 100
) -> Tuple[NearestNeighbors, Dict]:
    """
    Build a K-NN index for item embeddings.
    
    Args:
        item_embeddings: Item embedding matrix (num_items, embedding_dim)
        item_mappings: Dictionary mapping item indices to parent_asin
        n_neighbors: Number of neighbors to retrieve
    
    Returns:
        Tuple of (knn_model, idx_to_item_dict)
    """
    # Build K-NN model
    knn = NearestNeighbors(
        n_neighbors=min(n_neighbors + 1, len(item_embeddings)),  # +1 to exclude self
        metric='cosine',  # Use cosine similarity for embeddings
        algorithm='brute'  # Brute force for exact results
    )
    
    knn.fit(item_embeddings)
    
    # Create reverse mapping: embedding index -> parent_asin
    idx_to_item = {idx: item for item, idx in item_mappings.items()}
    
    print(f"K-NN index built with {len(item_embeddings)} items")
    
    return knn, idx_to_item


def retrieve_top_k_candidates(
    user_id: str,
    user_embeddings: np.ndarray,
    item_embeddings: np.ndarray,
    knn_model: NearestNeighbors,
    user_mappings: Dict,
    idx_to_item: Dict,
    k: int = 100,
    exclude_interacted: bool = True,
    user_interactions: pd.DataFrame = None
) -> List[str]:
    """
    Retrieve top-K candidate items for a given user using FM embeddings.
    
    Strategy: Compute user-item similarity scores by taking dot product of
    user embedding with all item embeddings, then select top-K.
    
    Args:
        user_id: User ID to get recommendations for
        user_embeddings: User embedding matrix
        item_embeddings: Item embedding matrix
        knn_model: Trained K-NN model (alternative method, not used in current implementation)
        user_mappings: Dictionary mapping user_id to embedding index
        idx_to_item: Dictionary mapping embedding index to parent_asin
        k: Number of candidates to retrieve
        exclude_interacted: Whether to exclude items user has already interacted with
        user_interactions: DataFrame with user's past interactions
    
    Returns:
        List of parent_asin values (top-K candidate items)
    """
    # Get user embedding
    if user_id not in user_mappings:
        raise ValueError(f"User {user_id} not found in embeddings")
    
    user_idx = user_mappings[user_id]
    user_embedding = user_embeddings[user_idx]  # Shape: (embedding_dim,)
    
    # Compute similarity scores: dot product of user embedding with all item embeddings
    # This gives us a score for each item indicating how well it matches the user
    similarity_scores = np.dot(item_embeddings, user_embedding)  # Shape: (num_items,)
    
    # Get top-K item indices (highest similarity scores)
    top_k_indices = np.argsort(similarity_scores)[::-1][:k+1]  # +1 in case we need to exclude one
    
    # Convert indices to parent_asin
    candidate_items = []
    for idx in top_k_indices:
        if idx in idx_to_item:
            candidate_items.append(idx_to_item[idx])
    
    # Exclude items user has already interacted with
    if exclude_interacted and user_interactions is not None:
        user_items = set(user_interactions[
            user_interactions['user_id'] == user_id
        ]['parent_asin'].unique())
        candidate_items = [item for item in candidate_items if item not in user_items]
    
    # Return top K (may be less if we excluded interactions)
    return candidate_items[:k]


## 3. Ranking Stage: XGBoost with Feature Store Metadata

### 3.1 Fetch Metadata from Online Feature Store


In [ ]:
def fetch_feature_store_metadata(
    candidate_items: List[str],
    user_id: str,
    item_feature_group: FeatureGroup,
    user_feature_group: FeatureGroup,
    point_in_time: datetime = None
) -> pd.DataFrame:
    """
    Fetch rich metadata from Online Feature Store for candidate items.
    
    This function performs point-in-time accurate joins to get:
    - Item features: Price, Category, Average Rating, User Rating Count
    - User features: User rating count, etc.
    
    Args:
        candidate_items: List of parent_asin values
        user_id: User ID for user features
        item_feature_group: Item Feature Group object
        user_feature_group: User Feature Group object
        point_in_time: Point in time for feature retrieval (default: now)
    
    Returns:
        DataFrame with enriched features for each candidate item
    """
    if point_in_time is None:
        point_in_time = datetime.now()
    
    enriched_features = []
    
    # Fetch user features
    user_features = None
    try:
        user_record = featurestore_runtime.get_record(
            FeatureGroupName=user_feature_group.name,
            RecordIdentifierValueAsString=user_id
        )
        if 'Record' in user_record:
            user_features = {
                feat['FeatureName']: feat['ValueAsString']
                for feat in user_record['Record']
            }
    except Exception as e:
        print(f"Warning: Could not fetch user features for {user_id}: {e}")
    
    # Fetch item features for each candidate
    for parent_asin in candidate_items:
        try:
            item_record = featurestore_runtime.get_record(
                FeatureGroupName=item_feature_group.name,
                RecordIdentifierValueAsString=parent_asin
            )
            
            if 'Record' in item_record:
                item_features = {
                    feat['FeatureName']: feat['ValueAsString']
                    for feat in item_record['Record']
                }
                
                # Combine user and item features
                combined_features = {
                    'parent_asin': parent_asin,
                    'user_id': user_id
                }
                
                # Add item features (handle different possible column names)
                if 'price' in item_features:
                    try:
                        combined_features['item_price'] = float(item_features.get('price', 0))
                    except:
                        combined_features['item_price'] = 0.0
                else:
                    combined_features['item_price'] = 0.0
                    
                if 'main_category' in item_features:
                    combined_features['item_category'] = item_features.get('main_category', '')
                else:
                    combined_features['item_category'] = ''
                    
                if 'average_rating' in item_features:
                    try:
                        combined_features['item_avg_rating'] = float(item_features.get('average_rating', 0))
                    except:
                        combined_features['item_avg_rating'] = 0.0
                else:
                    combined_features['item_avg_rating'] = 0.0
                    
                # Handle rating count (could be named differently)
                rating_count = 0.0
                for key in ['rating_count', 'user_rating_count', 'total_ratings']:
                    if key in item_features:
                        try:
                            rating_count = float(item_features.get(key, 0))
                            break
                        except:
                            pass
                combined_features['item_rating_count'] = rating_count
                
                # Add user features
                if user_features:
                    if 'rating_count_by_user' in user_features:
                        try:
                            combined_features['user_rating_count'] = float(
                                user_features.get('rating_count_by_user', 0)
                            )
                        except:
                            combined_features['user_rating_count'] = 0.0
                    else:
                        combined_features['user_rating_count'] = 0.0
                else:
                    combined_features['user_rating_count'] = 0.0
                
                enriched_features.append(combined_features)
                
        except Exception as e:
            print(f"Warning: Could not fetch features for item {parent_asin}: {e}")
            continue
    
    if not enriched_features:
        raise ValueError("No features retrieved from Feature Store")
    
    # Convert to DataFrame
    features_df = pd.DataFrame(enriched_features)
    
    # Fill missing values
    numeric_cols = features_df.select_dtypes(include=[np.number]).columns
    features_df[numeric_cols] = features_df[numeric_cols].fillna(0)
    
    return features_df


### 3.2 Prepare Training Data for XGBoost Ranker


In [ ]:
def prepare_ranking_training_data(
    interactions_df: pd.DataFrame,
    user_feature_group: FeatureGroup,
    item_feature_group: FeatureGroup,
    sample_size: int = None
) -> pd.DataFrame:
    """
    Prepare training data for XGBoost ranking model.
    
    For each user-item interaction, fetch features from Feature Store
    at the point-in-time of the interaction.
    
    Args:
        interactions_df: DataFrame with user_id, parent_asin, rating, event_time_seconds
        user_feature_group: User Feature Group
        item_feature_group: Item Feature Group
        sample_size: Optional sample size for faster training (None = use all)
    
    Returns:
        DataFrame with features and target (rating)
    """
    # Sample if needed
    if sample_size and len(interactions_df) > sample_size:
        interactions_df = interactions_df.sample(n=sample_size, random_state=42)
    
    print(f"Preparing ranking training data for {len(interactions_df)} interactions...")
    
    # Use Feature Store Dataset Builder for point-in-time joins
    builder = feature_store.create_dataset(
        base=interactions_df,
        event_time_identifier_feature_name='event_time_seconds',
        record_identifier_feature_name='user_id',
        output_path=f"s3://{bucket}/ranking-training-datasets/"
    )
    
    # Join with Feature Groups
    builder = builder.with_feature_group(
        feature_group=user_feature_group
    )
    
    builder = builder.with_feature_group(
        feature_group=item_feature_group
    )
    
    # Generate the dataset
    s3_uri, query = builder.to_csv_file()
    
    print(f"Ranking training dataset created at: {s3_uri}")
    
    # Load the dataset
    ranking_df = pd.read_csv(s3_uri)
    
    # Clean column names (remove prefixes added by Feature Store)
    ranking_df.columns = ranking_df.columns.str.replace(r'.*users.*\.', '', regex=True)
    ranking_df.columns = ranking_df.columns.str.replace(r'.*items.*\.', '', regex=True)
    
    # Ensure rating is the target
    if 'rating' not in ranking_df.columns:
        raise ValueError("Rating column not found in training data")
    
    print(f"Training data shape: {ranking_df.shape}")
    print(f"Features: {[col for col in ranking_df.columns if col != 'rating']}")
    
    return ranking_df


### 3.3 Train XGBoost Ranking Model


In [ ]:
def train_xgboost_ranker(
    training_df: pd.DataFrame,
    target_column: str = 'rating',
    instance_type: str = 'ml.m5.xlarge'
) -> Tuple[sagemaker.estimator.Estimator, List[str]]:
    """
    Train an XGBoost model for ranking candidate items.
    
    Args:
        training_df: DataFrame with features and target
        target_column: Name of target column
        instance_type: SageMaker instance type
    
    Returns:
        Tuple of (trained XGBoost estimator, feature_names)
    """
    # Separate features and target
    feature_columns = [col for col in training_df.columns if col != target_column]
    
    # Remove ID columns from features
    feature_columns = [col for col in feature_columns if col not in ['user_id', 'parent_asin', 'event_time_seconds', 'calendar_date']]
    
    X = training_df[feature_columns].copy()
    y = training_df[target_column].copy()
    
    # Handle categorical features (one-hot encode)
    categorical_cols = X.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
    
    # Fill missing values
    X = X.fillna(0)
    
    # Convert to CSV format for XGBoost (label in first column)
    training_data = pd.concat([y, X], axis=1)
    
    # Save to local file
    local_file = '/tmp/xgboost_training.csv'
    training_data.to_csv(local_file, index=False, header=False)
    
    # Upload to S3
    s3_key = 'xgboost-ranking/training_data.csv'
    s3_client = boto_session.client('s3')
    s3_client.upload_file(local_file, bucket, s3_key)
    
    training_s3_uri = f"s3://{bucket}/{s3_key}"
    
    # Save feature names for inference
    feature_names_key = 'xgboost-ranking/feature_names.json'
    feature_names_json = json.dumps(list(X.columns))
    s3_client.put_object(
        Bucket=bucket,
        Key=feature_names_key,
        Body=feature_names_json.encode('utf-8')
    )
    
    # Get XGBoost container
    container = image_uris.retrieve(
        "xgboost",
        boto_session.region_name,
        version='1.7-1'
    )
    
    # Create estimator
    xgb_estimator = sagemaker.estimator.Estimator(
        container,
        role=role_arn,
        instance_count=1,
        instance_type=instance_type,
        output_path=f"s3://{bucket}/xgboost-model-artifacts/",
        sagemaker_session=sagemaker_session
    )
    
    # Set hyperparameters
    xgb_estimator.set_hyperparameters(
        objective='reg:squarederror',  # Regression for rating prediction
        num_round=100,
        max_depth=6,
        eta=0.3,
        gamma=0,
        min_child_weight=1,
        subsample=0.8,
        silent=0
    )
    
    # Start training
    print("Starting XGBoost training...")
    xgb_estimator.fit({'train': training_s3_uri})
    
    print(f"XGBoost training complete! Model artifacts: {xgb_estimator.model_data}")
    
    return xgb_estimator, list(X.columns)


## 4. Inference Pipeline: Complete Recommendation Function


In [ ]:
class TwoStageRecommender:
    """
    Two-Stage Recommendation System:
    1. Retrieval: FM embeddings + K-NN
    2. Ranking: XGBoost with Feature Store metadata
    """
    
    def __init__(
        self,
        user_embeddings: np.ndarray,
        item_embeddings: np.ndarray,
        knn_model: NearestNeighbors,
        user_mappings: Dict,
        idx_to_item: Dict,
        xgb_predictor: sagemaker.predictor.Predictor,
        feature_names: List[str],
        user_feature_group: FeatureGroup,
        item_feature_group: FeatureGroup,
        user_interactions: pd.DataFrame = None
    ):
        self.user_embeddings = user_embeddings
        self.item_embeddings = item_embeddings
        self.knn_model = knn_model
        self.user_mappings = user_mappings
        self.idx_to_item = idx_to_item
        self.xgb_predictor = xgb_predictor
        self.feature_names = feature_names
        self.user_feature_group = user_feature_group
        self.item_feature_group = item_feature_group
        self.user_interactions = user_interactions
    
    def get_recommendations(
        self,
        user_id: str,
        top_k: int = 10,
        retrieval_k: int = 100
    ) -> List[Dict[str, any]]:
        """
        Get top-K recommendations for a user.
        
        Pipeline:
        1. Retrieve top-100 candidates using FM embeddings
        2. Enrich candidates with Feature Store metadata
        3. Rank candidates using XGBoost
        4. Return top-K items
        
        Args:
            user_id: User ID
            top_k: Number of final recommendations to return
            retrieval_k: Number of candidates to retrieve in stage 1
        
        Returns:
            List of dictionaries with 'parent_asin' and 'predicted_rating'
        """
        # Stage 1: Retrieval - Get top-K candidates
        print(f"Stage 1: Retrieving {retrieval_k} candidates for user {user_id}...")
        candidate_items = retrieve_top_k_candidates(
            user_id=user_id,
            user_embeddings=self.user_embeddings,
            item_embeddings=self.item_embeddings,
            knn_model=self.knn_model,
            user_mappings=self.user_mappings,
            idx_to_item=self.idx_to_item,
            k=retrieval_k,
            exclude_interacted=True,
            user_interactions=self.user_interactions
        )
        
        print(f"Retrieved {len(candidate_items)} candidates")
        
        if len(candidate_items) == 0:
            return []
        
        # Stage 2: Enrich with Feature Store metadata
        print(f"Stage 2: Enriching candidates with Feature Store metadata...")
        enriched_features = fetch_feature_store_metadata(
            candidate_items=candidate_items,
            user_id=user_id,
            item_feature_group=self.item_feature_group,
            user_feature_group=self.user_feature_group
        )
        
        print(f"Enriched {len(enriched_features)} candidates")
        
        # Stage 3: Rank with XGBoost
        print(f"Stage 3: Ranking candidates with XGBoost...")
        
        # Prepare features for XGBoost (match training format)
        feature_data = enriched_features.copy()
        
        # Handle categorical features (one-hot encode)
        categorical_cols = feature_data.select_dtypes(include=['object']).columns
        categorical_cols = [col for col in categorical_cols if col not in ['user_id', 'parent_asin']]
        
        if len(categorical_cols) > 0:
            feature_data = pd.get_dummies(feature_data, columns=categorical_cols, drop_first=True)
        
        # Ensure all training features are present
        for feat in self.feature_names:
            if feat not in feature_data.columns:
                feature_data[feat] = 0
        
        # Select only features used in training (in same order)
        X = feature_data[self.feature_names]
        
        # Fill missing values
        X = X.fillna(0)
        
        # Convert to CSV format for XGBoost (no header, no label column)
        csv_data = X.to_csv(index=False, header=False)
        
        # Get predictions
        predictions = self.xgb_predictor.predict(csv_data)
        
        # Parse predictions (XGBoost returns CSV string)
        if isinstance(predictions, bytes):
            predictions = predictions.decode('utf-8')
        
        pred_scores = [float(x) for x in predictions.strip().split('\n') if x.strip()]
        
        # Combine with item IDs
        results = [
            {
                'parent_asin': item,
                'predicted_rating': score
            }
            for item, score in zip(candidate_items[:len(pred_scores)], pred_scores)
        ]
        
        # Sort by predicted rating (descending)
        results.sort(key=lambda x: x['predicted_rating'], reverse=True)
        
        # Return top-K
        return results[:top_k]


## 5. Complete Workflow: Training and Deployment

Execute the following cells in order to train both models and set up the inference pipeline.


In [ ]:
# Step 1: Load interaction data
interactions_df = pd.read_parquet(
    "s3://recommendation-project-rapid/processed/all_beauty_dataset/",
    engine="pyarrow"
)

# Select only user_id, parent_asin, rating for FM training
fm_training_df = interactions_df[['user_id', 'parent_asin', 'rating']].copy()

print(f"Loaded {len(fm_training_df)} interactions")
print(f"Unique users: {fm_training_df['user_id'].nunique()}")
print(f"Unique items: {fm_training_df['parent_asin'].nunique()}")


In [ ]:
# Step 2: Prepare and train Factorization Machines
training_data_s3_uri, mappings, feature_dim = prepare_fm_training_data(
    interactions_df=fm_training_df,
    output_s3_path=f"s3://{bucket}/fm-training/training_data.txt"
)

# Train FM model
fm_estimator = train_factorization_machines(
    training_data_s3_uri=training_data_s3_uri,
    feature_dim=feature_dim,
    num_factors=64,
    epochs=10
)


In [ ]:
# Step 3: Extract embeddings
user_embeddings, item_embeddings, embedding_dict = extract_embeddings(
    fm_estimator=fm_estimator,
    mappings=mappings,
    num_factors=64
)

# Build K-NN index
knn_model, idx_to_item = build_knn_index(
    item_embeddings=item_embeddings,
    item_mappings=mappings['item_to_idx'],
    n_neighbors=100
)


In [ ]:
# Step 4: Load Feature Groups
# IMPORTANT: Update these names with your actual Feature Group names
# You can find them by running: sagemaker_client.list_feature_groups()

user_fg = FeatureGroup(
    name=USER_FEATURE_GROUP_NAME,
    sagemaker_session=sagemaker_session
)

item_fg = FeatureGroup(
    name=ITEM_FEATURE_GROUP_NAME,
    sagemaker_session=sagemaker_session
)

# Verify Feature Groups exist
try:
    user_fg.describe()
    item_fg.describe()
    print("Feature Groups loaded successfully")
except Exception as e:
    print(f"Error loading Feature Groups: {e}")
    print("Please update USER_FEATURE_GROUP_NAME and ITEM_FEATURE_GROUP_NAME")
    print("\nTo find your Feature Group names, run:")
    print("response = sagemaker_client.list_feature_groups()")
    print("for fg in response['FeatureGroupSummaries']:")
    print("    print(f\"- {fg['FeatureGroupName']}\")")


In [ ]:
# Step 5: Prepare ranking training data
ranking_training_df = prepare_ranking_training_data(
    interactions_df=interactions_df,
    user_feature_group=user_fg,
    item_feature_group=item_fg,
    sample_size=10000  # Use sample for faster training (remove for full dataset)
)


In [ ]:
# Step 6: Train XGBoost ranker
xgb_estimator, feature_names = train_xgboost_ranker(
    training_df=ranking_training_df,
    target_column='rating'
)


In [ ]:
# Step 7: Deploy XGBoost model
xgb_predictor = xgb_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer()
)

print(f"XGBoost endpoint deployed: {xgb_predictor.endpoint_name}")


In [ ]:
# Step 8: Initialize Two-Stage Recommender
recommender = TwoStageRecommender(
    user_embeddings=user_embeddings,
    item_embeddings=item_embeddings,
    knn_model=knn_model,
    user_mappings=mappings['user_to_idx'],
    idx_to_item=idx_to_item,
    xgb_predictor=xgb_predictor,
    feature_names=feature_names,
    user_feature_group=user_fg,
    item_feature_group=item_fg,
    user_interactions=interactions_df
)

print("Two-Stage Recommender initialized!")


In [ ]:
# Step 9: Test the recommendation pipeline
test_user_id = interactions_df['user_id'].iloc[0]

recommendations = recommender.get_recommendations(
    user_id=test_user_id,
    top_k=10,
    retrieval_k=100
)

print(f"\nTop 10 Recommendations for User: {test_user_id}")
print("=" * 60)
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. Item: {rec['parent_asin']}, Predicted Rating: {rec['predicted_rating']:.3f}")


## 6. Standalone Inference Function

For production use, here's a standalone function that can be called directly:


In [ ]:
def get_recommendations(
    user_id: str,
    recommender: TwoStageRecommender,
    top_k: int = 10
) -> List[Dict[str, any]]:
    """
    Standalone function to get recommendations for a user.
    
    This is the main entry point for the two-stage recommendation system.
    It orchestrates:
    (a) Retrieve 100 candidates via FM embeddings
    (b) Enrich them with Feature Store metadata
    (c) Rank them with XGBoost model
    (d) Return the top 10 products
    
    Args:
        user_id: User ID to get recommendations for
        recommender: Initialized TwoStageRecommender instance
        top_k: Number of recommendations to return (default: 10)
    
    Returns:
        List of recommendation dictionaries with 'parent_asin' and 'predicted_rating'
    
    Example:
        >>> recommendations = get_recommendations(
        ...     user_id="AGKHLEW2SOWHNMFQI...",
        ...     recommender=recommender,
        ...     top_k=10
        ... )
        >>> for rec in recommendations:
        ...     print(f"Item: {rec['parent_asin']}, Score: {rec['predicted_rating']:.2f}")
    """
    return recommender.get_recommendations(user_id=user_id, top_k=top_k)

# Example usage:
# recommendations = get_recommendations(
#     user_id="AGKHLEW2SOWHNMFQI...",
#     recommender=recommender,
#     top_k=10
# )


## Summary

This notebook implements a complete Two-Stage Recommendation System:

### Stage 1: Retrieval (Matrix Factorization)
- **Model**: SageMaker Factorization Machines
- **Input**: user_id and parent_asin interactions
- **Output**: User and Item embeddings (64-dimensional vectors)
- **Method**: K-Nearest Neighbors search to retrieve top-100 candidates

### Stage 2: Ranking (XGBoost)
- **Model**: SageMaker XGBoost
- **Input**: Top-100 candidates enriched with Feature Store metadata
  - Item features: Price, Category, Average Rating, Rating Count
  - User features: User Rating Count
- **Output**: Predicted ratings for each candidate
- **Method**: Point-in-time accurate feature joins from Online Feature Store

### Inference Pipeline
The `get_recommendations()` function orchestrates the complete flow:
1. Retrieves 100 candidates using FM embeddings
2. Enriches candidates with real-time Feature Store metadata
3. Ranks candidates using XGBoost
4. Returns top-10 products

### Key Features
- ✅ Point-in-time accurate feature joins
- ✅ Handles missing features gracefully
- ✅ Excludes already-interacted items
- ✅ Production-ready with proper error handling
- ✅ Uses SageMaker managed training and deployment
